In [ ]:
import SDS_Environment as SDS
from SDS_Environment import np, plt, tf, Camera

# %matplotlib notebook

In [3]:
def initializer(env, N):
    # ---------------------------------------------------------------------------------------
    camera = Camera(plt.figure(figsize=(15,15)))
    
    # ---------------------------------------------------------------------------------------
    # build models
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    np.random.seed(42)

    input_shape = [3]              # == env.observation_space.shape
    n_outputs = 2                  # == env.action_space.n

    model = []                     # make a model for each agent
    for i in range(N):
        model.append(tf.keras.models.Sequential([
            tf.keras.layers.Dense(32, activation="elu", input_shape=input_shape),
            tf.keras.layers.Dense(32, activation="elu"),
            tf.keras.layers.Dense(n_outputs, activation=custom_activation)
        ]))
        
        model[i] = tf.keras.models.load_model('100agent.keras', 
                            custom_objects={'custom_activation': tf.keras.layers.Activation(custom_activation)}, 
                            compile=False)
        
    # ---------------------------------------------------------------------------------------
    # initilaze states and rewards from environment class    
    state, reward = env.step(np.zeros(N), 1)
    state = [[state[j][i] for j in range(input_shape[0])] for i in range(N)]
    
    return camera, model, n_outputs, state
    
def custom_activation(x):
    xx = (tf.keras.backend.softplus(x))**0.5
    return -tf.keras.backend.elu(-xx + 100) + 100

In [7]:
def play_one_step(env, model, N, state, episode):
    action = np.zeros(env.N)

    while N < env.N:
        N += 1
        model.append(tf.keras.models.Sequential([
                    tf.keras.layers.Dense(32, activation="elu", input_shape=[len(state[0])]),
                    tf.keras.layers.Dense(32, activation="elu"),
                    tf.keras.layers.Dense(2, activation=custom_activation)
                ]))
                
        model[-1] = tf.keras.models.load_model('100agent.keras', 
                            custom_objects={'custom_activation': tf.keras.layers.Activation(custom_activation)}, 
                            compile=False)

    for i in range(env.N):
        actionnnn = model[i].predict(np.reshape(state[i], (3)).reshape(1, -1), verbose=0)[0]
        action[i] = np.argmax(actionnnn)
    
    next_state, reward = env.step((action-0.5)*2, episode)
    next_state = [[next_state[j][i] for j in range(3)] for i in range(env.N)]

    return next_state

In [8]:
def Plot_H_G(episode, Hamilton, Giant, Edges):
    plt.figure(figsize=(15,10))

    plt.subplot(2,1,1)
    plt.title("Hamilton")
    plt.plot(Hamilton)
    plt.text(0.7*episode, 1.1*max(Hamilton), "Min(H): %f" % (min(Hamilton)) )
    plt.text(0.7*episode, 1.2*max(Hamilton), "Arg(H): %f" % (np.argmin(Hamilton)) )
    plt.grid(alpha=0.3)

    plt.subplot(2,2,3)
    plt.title("Giant Component")
    plt.plot(Giant)
    plt.grid(alpha=0.3)

    plt.subplot(2,2,4)
    plt.title("Tansition Range")
    plt.plot(Edges)
    plt.grid(alpha=0.3) 

    plt.show()

In [ ]:
N = 100                      # Number of agents
L = 15                       # The length of the simulation box
env = SDS.Distributed_System(N,L)

camera, model, n_outputs, state = initializer(env, N)

Hamilton = []               # Just for plot total hamilton per episode
Giant  = []                 # Represents the percentage of members of the giant component 
Edges  = []
Energy = []


for episode in range(800):
    state = play_one_step(env, model, N, state, episode)
    

    # ---------------------------------------------------------------------------------------
    # Show Results
    H = 0
    for i in range(env.N): H += env.Hamiltonian(i)        # Hamiltonian of the whole system
    Hamilton.append(H) 
    Edges.append(env.k.sum()/2)
    Energy.append((0.2*env.r**2).sum())
    
    
    if episode%10 == 0:
        Giant.append( env.Plot(camera, episode) ) 

    print("\rEpisode: {}, H: {:.3f}, N: {:.1f}".format(episode, H, env.N), end="")
    
        
anim = camera.animate(interval= 120, repeat=True, repeat_delay= 500, blit=True)
anim.save('./result/animation_Learned.gif')

#-------------------------------------------------------------------------------
print("\n\n", np.average(Hamilton))
Plot_H_G(episode, Hamilton, Giant, Energy)

# Base Model

In [ ]:
N = 50                      # Number of agents
L = 8                       # The length of the simulation box
env = SDS.Distributed_System(N,L)

camera, model, n_outputs, state = initializer(env, N)

Hamilton = []               # Just for plot total hamilton per episode
Giant  = []                 # Represents the percentage of members of the giant component 
Edges  = []
Energy = []
R_average = []

for episode in range(300):
    action = np.zeros(N)
    for i in range(env.N):
        if env.k[i] < 5:
            action[i] = np.random.randint(2)
    
    next_state, reward = env.step(action, episode)
    next_state = [[next_state[j][i] for j in range(3)] for i in range(env.N)]
    state = next_state
      

    # ---------------------------------------------------------------------------------------
    # Show Results
    H = 0
    for i in range(env.N): H += env.Hamiltonian(i)        # Hamiltonian of the whole system
    Hamilton.append(H) 
    # Edges.append(env.k.sum()/2)
    Energy.append((0.2*env.r**2).sum())
    R_average.append(np.average(env.r))
    
    if episode%1 == 0:
        Giant.append( env.Plot(camera, episode) )

    print("\rEpisode: {}, H: {:.3f}, N: {:.1f}".format(episode, H, env.N), end="")
        
        
anim = camera.animate(interval= 120, repeat=True, repeat_delay= 500, blit=True)
anim.save('./result/animation_C2.gif')

#-------------------------------------------------------------------------------
print("\n", min(Hamilton), np.argmin(Hamilton), R_average[np.argmin(Hamilton)])
Plot_H_G(episode, Hamilton, Giant, Energy)

# tests